## 時系列データセットの作成

In [5]:
import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.range(10)

# drop_remainder=Trueとすると、sizeが足りていない場合を除去する
dataset = dataset.window(size=5, shift=1, drop_remainder=True)

# Flatten the windows by putting its elements in a single batch
dataset = dataset.flat_map(lambda window: window.batch(5))

# Print the results
for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [6]:
dataset = dataset.map(lambda window: (window[:-1], window[-1]))
# buffer_sizeはdatasetの要素数以上に設定する
dataset = dataset.shuffle(buffer_size=10)

for x,y in dataset:
    print(f"x = {x.numpy()}, y = {y.numpy()}")

x = [0 1 2 3], y = 4
x = [3 4 5 6], y = 7
x = [4 5 6 7], y = 8
x = [5 6 7 8], y = 9
x = [1 2 3 4], y = 5
x = [2 3 4 5], y = 6


In [7]:
dataset = dataset.batch(2).prefetch(1)

for x, y in dataset:
    print(f"x = {x.numpy()}, y = {y.numpy()}")

x = [[3 4 5 6]
 [1 2 3 4]], y = [7 5]
x = [[5 6 7 8]
 [4 5 6 7]], y = [9 8]
x = [[2 3 4 5]
 [0 1 2 3]], y = [6 4]


In [8]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

## 時系列学習サンプル

In [ ]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, input_shape=[window_size])
    # tf.keras.layers.Dense(10, input_shape=[window_size], activation="relu"),
    # tf.keras.layers.Dense(10, activation="relu"),
    # tf.keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9))
model.fit(dataset,epochs=100)

# predict
forecast = []
for time in range(len(series) - window_size):
    forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

## 学習率の最適化

`callbacks`を使って、学習率を最適化する。
学習率の変化によるlossの推移を見てlossが最小になる学習率を探す。
最適だと思われる学習率を使って学習を行う。

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
history = model.fit(dataset, epochs=100, callbacks=[lr_schedule])

In [ ]:
import matplotlib.pyplot as plt

lrs = 1e-8 * (10 ** (np.arange(100) / 20))
plt.figure(figsize=(10, 6))
plt.grid(True)
plt.semilogx(lrs, history.history["loss"])
plt.tick_params('both', length=10, width=1, which='both')
plt.axis([1e-8, 1e-3, 0, 50])